In [1]:
import pyspark
from pyspark.sql import SparkSession
#生成SparkSession实例
spark = SparkSession.builder \
     .master("local[*]") \
     .appName("test") \
     .config("spark.some.config.option", "some-value") \
     .getOrCreate()

## 1、df创建

### 1.1 读取文件

In [2]:
df1 = spark.read.csv("file:///home/ian/code/data/sparkml/doc_class.dat", sep='|', header=True)
df1.show(5)

+--------+----------+--------+--------------------+--------------------+
|myapp_id|typenameid|typename|          myapp_word|      myapp_word_all|
+--------+----------+--------+--------------------+--------------------+
| 1376533|         2|  action|game, android, world|game, android, wo...|
| 1376542|         2|  action|                game|game, app, enjoy,...|
| 1376603|         2|  action|run, tap, collect...|run, tap, collect...|
| 1376792|         2|  action|                 run|run, ath, game, m...|
| 1376941|         2|  action|fight, game, play...|fight, game, play...|
+--------+----------+--------+--------------------+--------------------+
only showing top 5 rows



### 1.2 list

In [4]:
df2 = spark.createDataFrame([[1, 2, 3],[4, 5, 6]], ["col1", "col2", "col0"])
df2.show()

+----+----+----+
|col1|col2|col0|
+----+----+----+
|   1|   2|   3|
|   4|   5|   6|
+----+----+----+



In [21]:
l = [('Alice', 18),('joey',27)]

In [22]:
spark.createDataFrame(l).collect()

[Row(_1='Alice', _2=18), Row(_1='joey', _2=27)]

In [23]:
spark.createDataFrame(l, ['name', 'age']).collect()

[Row(name='Alice', age=18), Row(name='joey', age=27)]

### 1.3 dict

In [24]:
d = [{'name': 'Alice', 'age': 18},{'name': 'Joey', 'age': 27}]
spark.createDataFrame(d).collect()

/home/ian/installed/anaconda3/lib/python3.6/site-packages/pyspark-2.3.1-py3.6.egg/pyspark/sql/session.py:340: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


[Row(age=18, name='Alice'), Row(age=27, name='Joey')]

### 1.4 rdd
两种方式：  
* createDataFrame  
* rdd.toDF

In [26]:
rdd = spark.sparkContext.parallelize(l)
spark.createDataFrame(rdd).collect()

[Row(_1='Alice', _2=18), Row(_1='joey', _2=27)]

In [30]:
df = spark.createDataFrame(rdd, ['name', 'age'])
df.collect()

[Row(name='Alice', age=18), Row(name='joey', age=27)]

In [28]:
from pyspark.sql import Row
Person = Row('name', 'age')
person = rdd.map(lambda r: Person(*r))
df2 = spark.createDataFrame(person)
df2.collect()

[Row(name='Alice', age=18), Row(name='joey', age=27)]

In [44]:
person.toDF().show()

+-----+---+
| name|age|
+-----+---+
|Alice| 18|
| joey| 27|
+-----+---+



In [29]:
from pyspark.sql.types import *
schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True)])
df3 = spark.createDataFrame(rdd, schema)
df3.collect()

[Row(name='Alice', age=18), Row(name='joey', age=27)]

In [31]:
df.toPandas()

,name,age
0,Alice,18
1,joey,27


In [32]:
spark.createDataFrame(df.toPandas()).collect()

[Row(name='Alice', age=18), Row(name='joey', age=27)]

In [35]:
import pandas
pandas.DataFrame([[1, 2],[3,4]])

,0,1
0,1,2
1,3,4


In [36]:
spark.createDataFrame(pandas.DataFrame([[1, 2]])).collect()

[Row(0=1, 1=2)]

In [39]:
spark.createDataFrame(rdd, "a: string, b: int").collect()

[Row(a='Alice', b=18), Row(a='joey', b=27)]

In [40]:
rdd = rdd.map(lambda row: row[1])
spark.createDataFrame(rdd, "int").collect()

[Row(value=18), Row(value=27)]

In [47]:
spark.createDataFrame(rdd, "int").show()

+-----+
|value|
+-----+
|   18|
|   27|
+-----+



In [41]:
spark.createDataFrame(rdd, "boolean").collect()

Py4JJavaError: An error occurred while calling o666.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 71 in stage 67.0 failed 1 times, most recent failure: Lost task 71.0 in stage 67.0 (TID 2476, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/ian/installed/spark/python/lib/pyspark.zip/pyspark/worker.py", line 230, in main
    process()
  File "/home/ian/installed/spark/python/lib/pyspark.zip/pyspark/worker.py", line 225, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/ian/installed/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 372, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/ian/installed/spark/python/lib/pyspark.zip/pyspark/util.py", line 55, in wrapper
    return f(*args, **kwargs)
  File "/home/ian/installed/anaconda3/lib/python3.6/site-packages/pyspark-2.3.1-py3.6.egg/pyspark/sql

In [43]:
rdd1 = spark.sparkContext.parallelize([[1,2,3]])
df3 = spark.createDataFrame(rdd1)
df3.show()

+---+---+---+
| _1| _2| _3|
+---+---+---+
|  1|  2|  3|
+---+---+---+



### 1.5 range

In [7]:
df = spark.range(5)
df

DataFrame[id: bigint]

In [8]:
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



## 2 df常用函数

### 2.0 查看
show,head,first,take  
df行数：df.count()  

### 2.1 排序
orderBy和sort好像是等同的

In [16]:
df.sort('id',ascending=False).show()

+---+
| id|
+---+
|  4|
|  3|
|  2|
|  1|
|  0|
+---+



In [11]:
df.orderBy('id',ascending=False).show()

+---+
| id|
+---+
|  4|
|  3|
|  2|
|  1|
|  0|
+---+



In [12]:
df.orderBy(desc('id')).show()

NameError: name 'desc' is not defined

In [13]:
from pyspark.sql.functions import desc
df.orderBy(desc('id')).show()

+---+
| id|
+---+
|  4|
|  3|
|  2|
|  1|
|  0|
+---+



In [14]:
df.orderBy(df['id'].desc()).show()

+---+
| id|
+---+
|  4|
|  3|
|  2|
|  1|
|  0|
+---+



In [15]:
df.orderBy(df.id.desc()).show()

+---+
| id|
+---+
|  4|
|  3|
|  2|
|  1|
|  0|
+---+



### 描述性统计

In [18]:
df.describe().show()

+-------+------------------+
|summary|                id|
+-------+------------------+
|  count|                 5|
|   mean|               2.0|
| stddev|1.5811388300841898|
|    min|                 0|
|    max|                 4|
+-------+------------------+



In [50]:
df1 = df.cache()
df1

DataFrame[name: string, age: bigint]

In [49]:
df.persist()

DataFrame[name: string, age: bigint]